In [ ]:
import pandas as pd
import numpy as np

In [34]:
# Read in GROUPED data
url = 'https://crime-bucket.s3.amazonaws.com/all_grouped_data.csv'
df = pd.read_csv(url)
df = df.drop('Unnamed: 0', axis=1)
df.head()

,Crm Cd,Crm Cd Desc,AREA NAME,Vict Age,Vict Sex,Vict Descent,Premis Desc,LAT,LON,SEASON,day_night
0,900,VIOLATION OF COURT ORDER,Newton,48,M,H,SINGLE FAMILY DWELLING,33.9825,-118.2695,winter,day
1,900,VIOLATION OF COURT ORDER,Hollywood,47,F,W,STREET,34.1016,-118.3295,winter,night
2,5000,VEHICLE-RELATED,Central,46,M,H,STREET,34.0389,-118.2643,winter,night
3,2000,ASSAULT/BATTERY,Central,51,M,B,OTHER PREMISE,34.0435,-118.2427,winter,night
4,2000,ASSAULT/BATTERY,Central,30,M,H,PARKING LOT,34.0450,-118.2640,winter,night


In [ ]:
# Test on first 500K rows
df = df[0:500000]

In [36]:
from sklearn.model_selection import train_test_split  
X = df.drop('Crm Cd', axis=1)
X = df.drop('Crm Cd Desc',axis=1)
X.head()

,Crm Cd,AREA NAME,Vict Age,Vict Sex,Vict Descent,Premis Desc,LAT,LON,SEASON,day_night
0,900,Newton,48,M,H,SINGLE FAMILY DWELLING,33.9825,-118.2695,winter,day
1,900,Hollywood,47,F,W,STREET,34.1016,-118.3295,winter,night
2,5000,Central,46,M,H,STREET,34.0389,-118.2643,winter,night
3,2000,Central,51,M,B,OTHER PREMISE,34.0435,-118.2427,winter,night
4,2000,Central,30,M,H,PARKING LOT,34.0450,-118.2640,winter,night


In [ ]:
X = df.drop('Crm Cd', axis=1)
X = df.drop('Crm Cd Desc',axis=1)
y = df['Crm Cd'].values.reshape(-1,1)

In [38]:
X_encoded = pd.get_dummies(X,columns=['AREA NAME','Vict Sex','Vict Descent','Premis Desc','SEASON','day_night'])
print(X_encoded)

        Crm Cd  Vict Age  ...  day_night_day  day_night_night
0          900        48  ...              1                0
1          900        47  ...              0                1
2         5000        46  ...              0                1
3         2000        51  ...              0                1
4         2000        30  ...              0                1
...        ...       ...  ...            ...              ...
499995    1000        39  ...              1                0
499996     901        48  ...              1                0
499997    2000        26  ...              0                1
499998    1000        25  ...              1                0
499999    1000        40  ...              1                0

[500000 rows x 260 columns]


In [39]:
print(X_encoded.shape, y.shape)

(500000, 260) (500000, 1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, np.ravel(y), test_size=0.3)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(max_depth=15, random_state=122, n_estimators=500, max_features="auto")

In [43]:
y_train

array([ 740, 5000, 1000, ..., 5000, 5000, 4000])

In [44]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=122,
                       verbose=0, warm_start=False)

In [ ]:
rfc_pred = rfc.predict(X_test)
rfc_pred_NOTSTANDARDIZED = rfc.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [50]:
# print(confusion_matrix, y_test, rfc_pred)
print(classification_report(y_test, rfc_pred))
classification_report__NOTSTANDARDIZED = classification_report(y_test, rfc_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         740       1.00      0.44      0.61      7023
         745       0.95      0.36      0.52      8064
         761       0.00      0.00      0.00       622
         888       0.00      0.00      0.00      1028
         900       0.00      0.00      0.00      1747
         901       0.00      0.00      0.00      1162
         930       0.00      0.00      0.00      5614
         946       1.00      0.00      0.00      1604
         956       0.00      0.00      0.00      2300
        1000       0.70      1.00      0.83     55472
        2000       0.90      1.00      0.94     24995
        4000       0.99      0.67      0.80     10113
        5000       0.99      0.99      0.99     30256

    accuracy                           0.82    150000
   macro avg       0.50      0.34      0.36    150000
weighted avg       0.78      0.82      0.77    150000



In [51]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, rfc_pred))
accuracy_NOTSTANDARDIZED = metrics.accuracy_score(y_test, rfc_pred)

Accuracy: 0.8211266666666667


In [ ]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
rfc_stdized = RandomForestClassifier(max_depth=15, random_state=122, n_estimators=500, max_features="auto")

In [57]:
rfc_stdized.fit(X_train_scaled, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=122,
                       verbose=0, warm_start=False)

In [ ]:
rfc_pred_STANDARDIZED = rfc_stdized.predict(X_test_scaled)

In [62]:
print(classification_report(y_test, rfc_pred_STANDARDIZED))
classification_report_STANDARDIZED = classification_report(y_test, rfc_pred_STANDARDIZED)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         740       1.00      0.44      0.61      7023
         745       0.95      0.36      0.52      8064
         761       0.00      0.00      0.00       622
         888       0.00      0.00      0.00      1028
         900       0.00      0.00      0.00      1747
         901       0.00      0.00      0.00      1162
         930       0.00      0.00      0.00      5614
         946       1.00      0.00      0.00      1604
         956       0.00      0.00      0.00      2300
        1000       0.70      1.00      0.83     55472
        2000       0.90      1.00      0.94     24995
        4000       0.99      0.67      0.80     10113
        5000       0.99      0.99      0.99     30256

    accuracy                           0.82    150000
   macro avg       0.50      0.34      0.36    150000
weighted avg       0.78      0.82      0.77    150000



In [63]:
print("Accuracy:",metrics.accuracy_score(y_test, rfc_pred_STANDARDIZED))
accuracy_STANDARDIZED = metrics.accuracy_score(y_test, rfc_pred_STANDARDIZED)

Accuracy: 0.8211466666666667
